In [1]:
%cd '/Users/abhilashdhal/Projects/'

/Users/abhilashdhal/Projects


## 1. Imports

In [2]:
import gevent.monkey
gevent.monkey.patch_all(thread=False, select=False)
import pandas as pd
import numpy as np

import os
from importlib import reload
from flatten_json import flatten
from tqdm import tqdm 
import src.Engines.gdc_engine as gdc_engine
import src.Connectors.gcp_bigquery_utils as gcp_bigquery_utils
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
reload(gcp_bigquery_utils)
reload(gdc_engine)

<module 'src.Engines.gdc_engine' from '/Users/abhilashdhal/Projects/src/Engines/gdc_engine.py'>

## 1. Initialize gdc_engine to get data from GDC

In [3]:

params = {
    'files.experimental_strategy': 'RNA-Seq', 
    'data_type': 'Gene Expression Quantification'
}

gdc_eng_inst = gdc_engine.GDCEngine(**params)


## 2. Fetch data from GDC for single or multiple primary sites and upload to bigquery

In [ ]:

primary_sites = [
    'Esophagus'
]

# Initialize BigQueryUtils with your project
bq_utils = gcp_bigquery_utils.BigQueryUtils(project_id='rnaseqml')
table_id = 'rnaseqml.rnaseqexpression.expr_clustered'

schema = [
    bigquery.SchemaField("case_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("file_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("expr_unstr_count", "INTEGER", mode="REPEATED"),
    bigquery.SchemaField("tissue_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("sample_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("primary_site", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tissue_or_organ_of_origin", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("age_at_diagnosis", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("primary_diagnosis", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("race", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("gender", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("group_identifier", "INTEGER", mode="NULLABLE")
]

# Create table with partitioning and clustering
bq_utils.create_bigquery_table_with_schema(
    table_id=table_id, schema=schema, partition_field="group_identifier", clustering_fields=["primary_site", "tissue_type"]
)

for site in tqdm(primary_sites):
    json_object = rna_seq_DGE_data[site]

    # Load data into BigQuery
    job = bq_utils.load_json_data(json_object, schema, table_id)
    job.result()  # Wait for the job to complete

    print(f"Data for {site} loaded successfully.")